In [51]:
import random
import time
%run pravljenjeGrafova.ipynb

#Stvara nasumicnu pocetnu populaciju
def pocetnaPopulacija(size, brojCvoreva):
    l1 = [[random.choice([0, 1]) for _ in range(brojCvoreva)] for _ in range(0,size)]
    l2 = [[]]
    for l in l1:
        l2.append([str(index+1) for index, value in enumerate(l) if value == 1])

    filtriranaLista = [unutrasnja for unutrasnja in l2 if unutrasnja]  #Izbaci prazne liste
    return filtriranaLista

# Prilagodjenost (ocjenjuje prema broju cvorova, ukoliko nije slabo povezan dom skup vraca 1000)
def fitness(jedinka, graf):  
    if jelSlaboPovezan(jedinka.copy(), graf):
        return len(jedinka)
    else:
        return 1000  # Velika ocjena za neslabo povezane dom skupove

In [52]:
# Funkcija za provjeru slabog povezivanja
def nadjiSusjede (graf,cvor):
    return graf[cvor].copy()

def jelSlaboPovezan(dominirajuci,graf):

    if len(dominirajuci)==0:
        return False

    trcvor = dominirajuci[0]
    del dominirajuci[0]
    
    zaObici = []
    oznaceni = set()
    
    oznaceni.add(trcvor)
    zaObici = nadjiSusjede(graf,trcvor)

    for l in zaObici:
            if l not in oznaceni:
                oznaceni.add(l)     

    while not len(graf)==len(oznaceni):
        if (len(zaObici)==0):
            break
        
        trcvor = zaObici[0]
        zaObici.remove(trcvor)

        #ako jeste dominirajuci
        if trcvor in dominirajuci:

            dominirajuci.remove(trcvor)          
            oznaceni.add(trcvor)
            lista1 = nadjiSusjede(graf,trcvor)

            for l in lista1:
                if  l not in oznaceni:
                    zaObici.append(l)
                oznaceni.add(l)     
        #ako nije dom onda uzimamo njegove susjede koji su dom i dodajemo ih za pretragu
        else:
            lista1 = nadjiSusjede(graf,trcvor)
            for l in lista1:
                if l in dominirajuci:
                    zaObici.append(l)

    #Ako smo izbrisali sve dominirajuce i ako su oznaceni svi cvorovi iz grafa, znaci da jeste slabo povezan DS
    if(len(dominirajuci)==0):
        if (len(oznaceni)==len(graf)):
            return True
        else:
            return False
    return False

In [53]:
#Operator ukrstanja
def ukrstanje(parent1, parent2):
    ukrstanje_point1 = random.randint(0, len(parent1))
    ukrstanje_point2 = random.randint(0, len(parent2))

    children = []
    child1 = parent1[:ukrstanje_point1] + parent2[ukrstanje_point2:]
    child1 = list(set(child1)) #Uklanjamo duplikate
    child2 = parent2[:ukrstanje_point1] + parent1[ukrstanje_point2:]
    child2 = list(set(child2)) #Uklanjamo duplikate

    children.append(child1)
    children.append(child2)

    return children

#Operator mutacije
def mutacija(jedinka,brC):
    for i in range(brC):
        if random.random() < vjvMutacije:         
            if (i+1) in jedinka:
                jedinka.remove(i+1)
    return jedinka

In [54]:
# Genetski algoritam i parametri

brojPopulacije = 50
vjvMutacije = 0.2

def genetskiAlgoritam(graf,vrijemeTrajanja,stop):

    brojCvoreva = len(graf)
    populacija = pocetnaPopulacija(brojPopulacije, brojCvoreva)
    print("Pocetna populacija",populacija)
    startTime = time.time()
    br=0
    trBroj=10000
    while True:
        
        populacija.sort(key=lambda jedinka: fitness(jedinka, graf))
        elitneJedinke = populacija[:2]
        najboljaJedinka=elitneJedinke[0]
        
        if len(najboljaJedinka) == trBroj:
            br+=1
        else:
            br=0
            trBroj=len(najboljaJedinka)
        if(br==stop):
            break
        
        fitnesOcjene = [1/fitness(jedinka1, graf) for jedinka1 in populacija]
        ostaleJedinke = random.choices(populacija[2:], weights=fitnesOcjene[2:],k=(brojPopulacije-2)//2)
        parents = elitneJedinke + ostaleJedinke

        novaGen = []
        for e in elitneJedinke:
            novaGen.append(e)

        for _ in range((brojPopulacije-2)//2):
            parent1, parent2 = random.sample(parents, 2)
            children = ukrstanje(parent1, parent2)
            
            for c in children:
                child = mutacija(c,brojCvoreva)
                novaGen.append(child)
            
        populacija = novaGen
        if (time.time() - startTime)>vrijemeTrajanja:
            break
        
    print("Populacija na kraju",populacija)
    return min(populacija, key=lambda s: fitness(s, graf))


In [55]:
#Grafovi

g1 = {
    1: [2, 3],
    2: [1,3,4],
    3: [1,2,4],
    4: [2,3,5,6],
    5: [4,6,7,8,9],
    6: [4,5],
    7: [5],
    8: [5],
    9: [5]
}

g2 = {
    1: [2, 3],
    2: [1,4,3],
    3: [1,2],
    4: [2]
}

g3 = {
    1: [2,3],
    2: [1,3,4,5],
    3: [1,2,20],
    4: [2,5],
    5: [2,4,6,7],
    6: [5,12],
    7: [5,8,9],
    8: [7,9],
    9: [7,8,10,14,15,16],
    10: [9,11,13,14,16],
    11: [10,12,13,14],
    12: [6,11],
    13: [10,11,14],
    14: [9,10,11,13,15,16],
    15: [9,10,14,16],
    16: [9,10,14,15,17],
    17: [16,18,19,20],
    18: [17,19,20],
    19: [17,18,20],
    20: [3,17,18,19]
}

g4 = {
    1: [2,3,4],
    2: [1,5,6],
    3: [1,7,8],
    4: [1,9,10],
    5: [2,10,11],
    6: [2,7,12],
    7: [3,6,13],
    8: [3,9,14],
    9: [4,8,15],
    10: [4,5,16],
    11: [5,12,17],
    12: [6,11,18],
    13: [7,14,18],
    14: [8,13,19],
    15: [9,16,19],
    16: [10,15,17],
    17: [11,16,20],
    18: [12,13,20],
    19: [14,15,20],
    20: [17,18,19]
}

g5 = {
    1: [4],
    2: [4],
    3: [4],
    4: [1,2,3,5], 
    5: [4,11], #5:[4,6]
    6: [7,11], #6:[5,7]
    7: [6,8,9,10],
    8: [7],
    9: [7],
    10: [7],
    11: [5,6] #
}

g6 = {
    1: [4],
    2: [4],
    3: [4],
    4: [1,2,3,5], 
    5: [4,6],
    6: [5,7],
    7: [6,8,9,10],
    8: [7],
    9: [7],
    10: [7],
}

In [56]:
# Pokretanje genetskog algoritma
startTime = time.time()
gen = genetskiAlgoritam(napraviGraf("veliki grafovi/p-hat1500-1.txt"),600,300)
print("Minimalni slabo povezani dominirajuci skup:", gen)
print("Duzina",len(gen))
print("--- %s sekundi ---" % (time.time() - startTime))

Pocetna populacija [['1', '3', '4', '6', '7', '8', '11', '12', '15', '16', '18', '20', '22', '23', '25', '26', '33', '36', '37', '38', '41', '43', '44', '45', '46', '47', '48', '50', '51', '52', '53', '55', '56', '57', '59', '64', '67', '69', '70', '73', '74', '76', '79', '80', '81', '82', '85', '86', '87', '88', '91', '92', '93', '95', '97', '99', '100', '101', '103', '105', '106', '115', '119', '120', '122', '124', '125', '127', '129', '132', '136', '137', '139', '140', '142', '143', '146', '147', '149', '154', '158', '159', '161', '163', '164', '165', '171', '172', '173', '176', '177', '178', '179', '181', '182', '186', '187', '188', '189', '190', '191', '194', '196', '199', '200', '201', '203', '206', '207', '212', '213', '215', '218', '219', '222', '223', '228', '230', '231', '232', '233', '234', '235', '237', '238', '239', '247', '249', '254', '256', '259', '260', '261', '264', '265', '266', '267', '268', '270', '271', '273', '278', '281', '282', '283', '284', '287', '291', '293'